Tady bude úloha, která spojí údaje o platech v krajích s politickou reprezentací v krajích

tady bude neco o uloze blablabla

In [1]:
// tady bude nacteni dependenci
kernel.silent(true)

In [2]:
import coursierapi.MavenRepository
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [3]:
import $ivy.`com.github.propi:rdfrules:1.5.0`
import collection._
import org.apache.jena.riot.Lang

import com.github.propi.rdfrules.data._
import com.github.propi.rdfrules.algorithm.amie._
import com.github.propi.rdfrules.algorithm.dbscan._
import com.github.propi.rdfrules.utils._
import com.github.propi.rdfrules.index._
import com.github.propi.rdfrules.rule._
import com.github.propi.rdfrules.ruleset._

import reflect.io._, Path._
import reflect.io._
import Path._
kernel.silent(false)

In [3]:
// tady bude nacteni a  predstaveni salaries nezpracovanych dat

In [3]:
// tady bude zpracovani salaries dat

In [4]:
val graphs = "../data/salaries".toDirectory.files.map(_.path).filter(g => g matches """.*\.ttl""")

graphs: Iterator[String] = non-empty iterator

In [5]:
var salariesCubesMap: Map[String, Graph] = Map()
graphs.foreach(g => {
    val name = g.replaceAll("\\.ttl", "").replaceAll("^.*/", "")
    salariesCubesMap = salariesCubesMap + (name -> Graph(g))
})

salariesCubesMap: Map[String, Graph] = Map(
  "salaries-female" -> com.github.propi.rdfrules.data.Graph@6782533c,
  "salaries-male" -> com.github.propi.rdfrules.data.Graph@13e2ffab,
  "salaries-total" -> com.github.propi.rdfrules.data.Graph@507901f9
)

In [6]:
import eu.easyminer.discretization.impl.Interval
import eu.easyminer.discretization.impl.IntervalBound._
val rounded = (value: Double, scale: Integer) => BigDecimal(value).setScale(scale, BigDecimal.RoundingMode.HALF_UP).toDouble
val intervalToString = (i: Interval) => "<"+rounded(i.minValue.value,2)+ "__"+rounded(i.maxValue.value,2)+")"

import eu.easyminer.discretization.impl.Interval

import eu.easyminer.discretization.impl.IntervalBound._

rounded: (Double, Integer) => Double = ammonite.$sess.cmd5$Helper$$Lambda$2963/0x0000000840ce6040@74ee9cbc
intervalToString: Interval => String = ammonite.$sess.cmd5$Helper$$Lambda$2964/0x0000000840ce6840@4bf33df3

In [7]:
val uri = (value: String) => TripleItem.Uri(value)
val averageSalary = uri("http://data.czso.cz/ontology/prumernaMzda")
val medianSalary = uri("http://data.czso.cz/ontology/medianMzdy")
val measureUris = Array(averageSalary,medianSalary)

uri: String => TripleItem.Uri = ammonite.$sess.cmd6$Helper$$Lambda$2976/0x0000000840cef040@609aa3a0
averageSalary: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/prumernaMzda"
)
medianSalary: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/medianMzdy"
)
measureUris: Array[TripleItem.Uri] = Array(
  LongUri("http://data.czso.cz/ontology/prumernaMzda"),
  LongUri("http://data.czso.cz/ontology/medianMzdy")
)

In [8]:
val equiFrequent = (intervalCount: Int) => DiscretizationTask.Equifrequency(intervalCount)

val discretizeEquifrequent = (graph: Graph, intervalsCount: Int, measure: TripleItem.Uri) => {
    graph.filter(t => t.predicate == measure)
         .discretize(equiFrequent(intervalsCount))(quad => quad.triple.predicate == measure)
}

val discretizeEquifrequentAndGetIntervals = (graph: Graph, intervalsCount: Int, measure: TripleItem.Uri) => {
    graph.filter(t => t.predicate == measure)
         .discretizeAndGetIntervals(equiFrequent(intervalsCount))(quad => quad.triple.predicate == measure)
}

equiFrequent: Int => DiscretizationTask.Equifrequency = ammonite.$sess.cmd7$Helper$$Lambda$3016/0x0000000840d0f840@766c41e6
discretizeEquifrequent: (Graph, Int, TripleItem.Uri) => Graph = ammonite.$sess.cmd7$Helper$$Lambda$3017/0x0000000840d0f440@ee5ad4
discretizeEquifrequentAndGetIntervals: (Graph, Int, TripleItem.Uri) => IndexedSeq[Interval] = ammonite.$sess.cmd7$Helper$$Lambda$3018/0x0000000840d10040@426b2935

In [9]:
import eu.easyminer.discretization.impl.Interval

val qbDataSetUri = "http://purl.org/linked-data/cube#dataSet";

val renameInterval = (t: Triple, intervals: IndexedSeq[Interval], suffix: String) => {
    var result = t
    intervals.foreach(interval => {
        val _interval = t.`object`.asInstanceOf[TripleItem.Interval]
        if (_interval.interval == interval) {
            result = t.copy(`object` = TripleItem.Uri(_interval.toString+"_"+suffix+"_"+(intervals.indexOf(interval)+1)+"/"+intervals.size) )
        }
    })
    result
}

val discretize = (graph: Graph, name: String, intervalCounts: Array[Int]) => {
    var result = Dataset() + 
    graph.filter(t => !measureUris.contains(t.predicate))
    .map(t => if (t.predicate.hasSameUriAs(qbDataSetUri)) t.copy(`object` = uri(name)) else t)
    
        measureUris.foreach(measure => {
            intervalCounts.foreach(intervalCount => {
                val intervals = discretizeEquifrequentAndGetIntervals(graph, intervalCount, measure)
                val measures = discretizeEquifrequent(graph, intervalCount, measure)
                result = result + measures.map(t => {renameInterval(t, intervals, "ef"+intervalCount)})
            })
    })
    result
}

import eu.easyminer.discretization.impl.Interval


qbDataSetUri: String = "http://purl.org/linked-data/cube#dataSet"
renameInterval: (Triple, IndexedSeq[Interval], String) => Triple = ammonite.$sess.cmd8$Helper$$Lambda$3073/0x0000000840d31c40@325ff2ef
discretize: (Graph, String, Array[Int]) => Dataset = ammonite.$sess.cmd8$Helper$$Lambda$3074/0x0000000840d32840@3ef8ad94

In [10]:
val intervalCounts = Array(2, 3, 5, 7)
var salaries = Dataset()
salariesCubesMap
.map(t => discretize(t._2,t._1,intervalCounts))
.foreach(d => salaries = salaries + d)

intervalCounts: Array[Int] = Array(2, 3, 5, 7)
salaries: Dataset = com.github.propi.rdfrules.data.Dataset@307594b0

In [10]:
// tady bude nacteni a predstaveni (i dotazy pro extrakci) wikidata dat

In [10]:
// tady bude zpracování wikidata dat

In [10]:
// TODO ukazat dotaz

In [34]:
val wikidata = Graph("wikidata","../data/wikidata/wikidata.ttl")
wikidata.size

wikidata: Graph = com.github.propi.rdfrules.data.Graph@2e5202e1
res33_1: Int = 23291

In [11]:
// TODO ukazat dotaz

In [35]:
val wikidataChronical = Graph("wikidata","../data/wikidata/wikidataChronicalDay.ttl")
wikidataChronical.size

wikidataChronical: Graph = com.github.propi.rdfrules.data.Graph@645fc4f5
res34_1: Int = 3268

In [36]:
val start = "http://www.wikidata.org/prop/P580"
val end = "http://www.wikidata.org/prop/P582"
val wikidataNoTime = wikidata.filter(t => !(t.predicate.hasSameUriAs(start) || t.predicate.hasSameUriAs(end)))

start: String = "http://www.wikidata.org/prop/P580"
end: String = "http://www.wikidata.org/prop/P582"
wikidataNoTime: Graph = com.github.propi.rdfrules.data.Graph@58506a7a

In [37]:
var wikidata = Dataset() + wikidataNoTime + wikidataChronical
wikidata = wikidata.setPrefixes(Traversable(
    Prefix("wd", "http://www.wikidata.org/entity/"),
    Prefix("p", "http://www.wikidata.org/prop/"),
    Prefix("prfx", "http://kizi.vse.cz/novp19/diploma-thesis/")
    )   
)
wikidata.size

wikidata: Dataset = com.github.propi.rdfrules.data.Dataset@f149848
res36_2: Int = 24937

In [14]:
// tady bude zmergovani dat

In [38]:
val czsoWikidataLinking = Dataset("../data/linking/czsoWikidataLinking.ttl")
czsoWikidataLinking.size
val refPeriodLinking = Dataset("../data/linking/cssaday-datagov.ttl")
refPeriodLinking.size

czsoWikidataLinking: Dataset = com.github.propi.rdfrules.data.Dataset@2e10a1e4
res37_1: Int = 90
refPeriodLinking: Dataset = com.github.propi.rdfrules.data.Dataset@16920480
res37_3: Int = 165

In [39]:
val dataset = salaries + refPeriodLinking + wikidata + czsoWikidataLinking
dataset.size

dataset: Dataset = com.github.propi.rdfrules.data.Dataset@dde69e3
res38_1: Int = 26830

In [40]:
val index = dataset.index()

index: Index = com.github.propi.rdfrules.index.Index$FromDatasetFullyPreservedIndex@78dace87

In [17]:
/*val appliesTo = uri("http://kizi.vse.cz/novp19/diploma-thesis/appliesToRefPeriod")
val wdProperty = (value: Int) => uri("http://www.wikidata.org/prop/P" + value)
val qbDataSet = uri(qbDataSetUri)

// salariesMeasures
val averageSalary = TripleItem.Uri("http://data.czso.cz/ontology/prumernaMzda")
val medianSalary = TripleItem.Uri("http://data.czso.cz/ontology/medianMzdy")
val oneOfSalariesMeasures = OneOf(averageSalary,medianSalary)

// czso dimensions
val czsoOntology = "http://data.czso.cz/ontology/"
val czsoRefPeriod = uri(czsoOntology+"refPeriod")
val czsoRefArea = uri(czsoOntology+"refArea")
val czsoSex = uri(czsoOntology+"sex")

val constantsAtObject = RuleConstraint.ConstantsAtPosition.ConstantsPosition.Object
val constantsOnlyAtObject = RuleConstraint.ConstantsAtPosition(constantsAtObject)

val salariesTotal = uri("salaries-total")*/

In [41]:
val czsoOntology = "http://data.czso.cz/ontology/"
val czsoRefArea = uri(czsoOntology+"refArea")
val czsoRefPeriod = uri(czsoOntology+"refPeriod")

val wdProperty = (value: Int) => uri("http://www.wikidata.org/prop/P" + value)
val appliesTo = uri("http://kizi.vse.cz/novp19/diploma-thesis/appliesToRefPeriod")

val averageSalary = TripleItem.Uri("http://data.czso.cz/ontology/prumernaMzda")
val medianSalary = TripleItem.Uri("http://data.czso.cz/ontology/medianMzdy")
val oneOfSalariesMeasures = OneOf(averageSalary,medianSalary)

val qbDataSet = uri(qbDataSetUri)

// val salariesTotal = uri("salaries-total") // OPRAVDU TO CHCI JENOM Z TOTALU ????


val salariesAlignmentPattern: RulePattern = (
    AtomPattern(subject = 'd', predicate = czsoRefArea, `object` = 'g') &:
    AtomPattern(subject = 'g', predicate = wdProperty(6), `object` = 'f') &:
    AtomPattern(subject = 'f', predicate = wdProperty(6), `object` = 'e') &:   
    AtomPattern(subject = 'e', predicate = wdProperty(102), `object` = 'b') &:
    AtomPattern(subject = 'd', predicate = oneOfSalariesMeasures, `object` = AnyConstant) &:
    AtomPattern(subject = 'd', predicate = qbDataSet, `object` = AnyConstant) &:
    
    AtomPattern(subject = 'd', predicate = czsoRefPeriod, `object` = 'c') &:
    AtomPattern(subject = 'b', predicate = appliesTo, `object` = 'c') &:
    AtomPattern(subject = 'b', predicate = wdProperty(102), `object` = 'a')
    =>:
    AtomPattern(subject = 'a', predicate = wdProperty(1387), `object` = AnyConstant)
)

czsoOntology: String = "http://data.czso.cz/ontology/"
czsoRefArea: TripleItem.Uri = LongUri("http://data.czso.cz/ontology/refArea")
czsoRefPeriod: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/refPeriod"
)
wdProperty: Int => TripleItem.Uri = ammonite.$sess.cmd40$Helper$$Lambda$3914/0x0000000840f5e840@36310f73
appliesTo: TripleItem.Uri = LongUri(
  "http://kizi.vse.cz/novp19/diploma-thesis/appliesToRefPeriod"
)
averageSalary: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/prumernaMzda"
)
medianSalary: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/medianMzdy"
)
oneOfSalariesMeasures: AtomPattern.AtomItemPattern.OneOf = OneOf(
  ArrayBuffer(
    Constant(LongUri("http://data.czso.cz/ontology/prumernaMzda")),
    Constant(LongUri("http://data.czso.cz/ontology/medianMzdy"))
  )
)
qbDataSet: TripleItem.Uri = LongUri("http://purl.org/linked-data/cube#dataSet")
salariesAlignmentPattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(
      Variable(Var

In [42]:
val constantsAtObject = RuleConstraint.ConstantsAtPosition.ConstantsPosition.Object
val constantsOnlyAtObject = RuleConstraint.ConstantsAtPosition(constantsAtObject)

val salariesAlignmentPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(10))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(1))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(salariesAlignmentPattern)

constantsAtObject: RuleConstraint.ConstantsAtPosition.ConstantsPosition.Object.type = Object
constantsOnlyAtObject: RuleConstraint.ConstantsAtPosition = ConstantsAtPosition(
  Object
)
salariesAlignmentPatternTask: com.github.propi.rdfrules.algorithm.RulesMining = com.github.propi.rdfrules.algorithm.amie.Amie@69c06454

In [43]:
val startTimeMillis = System.currentTimeMillis()
val salariesAlignmentPatternTaskRuleset = index.mine(salariesAlignmentPatternTask)
println("rules: "+salariesAlignmentPatternTaskRuleset.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

2021-10-27 08:43:47:272 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Predicates trimming.
2021-10-27 08:43:47:287 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Subjects indexing.
2021-10-27 08:43:47:307 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Subjects trimming.
2021-10-27 08:43:47:316 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Objects indexing.
2021-10-27 08:43:47:334 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Objects trimming.
2021-10-27 08:43:47:338 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=1,
MinHeadCoverage=0.0,
MinSupport=1,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=10,
WithConstants=true,
ConstantsPosition=Object,
Timeout=60000,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?d),Constant(Constant(1481837794)),Variable(?g),Any), 

rules: 251
duration: 2s


startTimeMillis: Long = 1635317026531L
salariesAlignmentPatternTaskRuleset: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@3d5fc9c8
endTimeMillis: Long = 1635317028715L
durationSeconds: Long = 2L

In [45]:
salariesAlignmentPatternTaskRuleset.export("../rulesets/salaries-wikidata/salariesAlignmentPatternTaskRuleset.txt")

In [46]:
salariesAlignmentPatternTaskRuleset
.filter(rule => rule.measures(Measure.Support).value > 1)
.computePcaConfidence(0.0)
.sortBy(Measure.PcaConfidence, Measure.Support)
.export("../rulesets/salaries-wikidata/salariesAlignmentPatternTaskRulesetFiltered.txt")

In [47]:
val salariesPartyPattern: RulePattern = (
    AtomPattern(subject = 'c', predicate = czsoRefArea, `object` = 'f') &:
    AtomPattern(subject = 'f', predicate = wdProperty(6), `object` = 'e') &:
    AtomPattern(subject = 'e', predicate = wdProperty(6), `object` = 'd') &:   
    AtomPattern(subject = 'd', predicate = wdProperty(102), `object` = 'a') &:
    AtomPattern(subject = 'c', predicate = oneOfSalariesMeasures, `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = qbDataSet, `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = czsoRefPeriod, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = appliesTo, `object` = 'b')
    =>:
    AtomPattern(subject = 'a', predicate = wdProperty(102), `object` = AnyConstant)
)

salariesPartyPattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(
      Variable(Variable(2)),
      Constant(LongUri("http://data.czso.cz/ontology/refArea")),
      Variable(Variable(5)),
      Any
    ),
    AtomPattern(
      Variable(Variable(5)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(4)),
      Any
    ),
    AtomPattern(
      Variable(Variable(4)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(3)),
      Any
    ),
    AtomPattern(
      Variable(Variable(3)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(0)),
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      OneOf(
        ArrayBuffer(
          Constant(LongUri("http://data.czso.cz/ontology/prumernaMzda")),
          Constant(LongUri("http://data.czso.cz/ontology/medianMzdy"))
        )
      ),
      AnyConstant,
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),


In [48]:
val salariesPartyPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(9))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(1))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(salariesPartyPattern)

salariesPartyPatternTask: com.github.propi.rdfrules.algorithm.RulesMining = com.github.propi.rdfrules.algorithm.amie.Amie@6ca53866

In [49]:
val startTimeMillis = System.currentTimeMillis()
val salariesPartyPatternTaskRuleset = index.mine(salariesPartyPatternTask)
println("rules: "+salariesPartyPatternTaskRuleset.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

2021-10-27 08:45:56:055 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=1,
MinHeadCoverage=0.0,
MinSupport=1,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=9,
WithConstants=true,
ConstantsPosition=Object,
Timeout=60000,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?c),Constant(Constant(1481837794)),Variable(?f),Any), Mapped(Variable(?f),Constant(Constant(774364698)),Variable(?e),Any), Mapped(Variable(?e),Constant(Constant(774364698)),Variable(?d),Any), Mapped(Variable(?d),Constant(Constant(52500897)),Variable(?a),Any), Mapped(Variable(?c),OneOf(ArrayBuffer(Constant(Constant(1433642368)), Constant(Constant(1396113393)))),AnyConstant,Any), Mapped(Variable(?c),Constant(Constant(624690160)),AnyConstant,Any), Mapped(Variable(?c),Constant(Constant(-901057765)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(217712947)),Variable(?b),Any)),Some(Mapped(Variable(?a),Constant(Constant(52500897)),AnyConst

rules: 296
duration: 0s


startTimeMillis: Long = 1635317156055L
salariesPartyPatternTaskRuleset: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@2c41d37e
endTimeMillis: Long = 1635317156891L
durationSeconds: Long = 0L

In [50]:
salariesPartyPatternTaskRuleset.export("../rulesets/salaries-wikidata/salariesPartyPatternTaskRuleset.txt")

In [52]:
salariesPartyPatternTaskRuleset
.filter(rule => rule.measures(Measure.Support).value > 1)
.computePcaConfidence(0.0)
.sortBy(Measure.PcaConfidence, Measure.Support)
.export("../rulesets/salaries-wikidata/salariesPartyPatternTaskRulesetFiltered.txt")

In [53]:
val alignmentSalariesPattern: RulePattern = (
    AtomPattern(subject = 'a', predicate = czsoRefArea, `object` = 'g') &:
    AtomPattern(subject = 'g', predicate = wdProperty(6), `object` = 'f') &:
    AtomPattern(subject = 'f', predicate = wdProperty(6), `object` = 'e') &:
    AtomPattern(subject = 'e', predicate = wdProperty(102), `object` = 'c') &:  
    AtomPattern(subject = 'd', predicate = wdProperty(1387)) &:
    AtomPattern(subject = 'c', predicate = wdProperty(102), `object` = 'd') &:
    AtomPattern(subject = 'c', predicate = appliesTo, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = czsoRefPeriod, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = qbDataSet, `object` = AnyConstant)
    =>:
    AtomPattern(subject = 'a', predicate = oneOfSalariesMeasures)
)

alignmentSalariesPattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(
      Variable(Variable(0)),
      Constant(LongUri("http://data.czso.cz/ontology/refArea")),
      Variable(Variable(6)),
      Any
    ),
    AtomPattern(
      Variable(Variable(6)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(5)),
      Any
    ),
    AtomPattern(
      Variable(Variable(5)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(4)),
      Any
    ),
    AtomPattern(
      Variable(Variable(4)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(2)),
      Any
    ),
    AtomPattern(
      Variable(Variable(3)),
      Constant(LongUri("http://www.wikidata.org/prop/P1387")),
      Any,
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(3)),
      Any
    ),
    AtomPattern(
...

In [54]:
val alignmentSalariesPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(10))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(1))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(alignmentSalariesPattern)

alignmentSalariesPatternTask: com.github.propi.rdfrules.algorithm.RulesMining = com.github.propi.rdfrules.algorithm.amie.Amie@75161a7f

In [55]:
val startTimeMillis = System.currentTimeMillis()
val alignmentSalariesPatternTaskRuleset = index.mine(alignmentSalariesPatternTask)
println("rules: "+alignmentSalariesPatternTaskRuleset.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

2021-10-27 08:49:49:626 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=1,
MinHeadCoverage=0.0,
MinSupport=1,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=10,
WithConstants=true,
ConstantsPosition=Object,
Timeout=60000,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?a),Constant(Constant(1481837794)),Variable(?g),Any), Mapped(Variable(?g),Constant(Constant(774364698)),Variable(?f),Any), Mapped(Variable(?f),Constant(Constant(774364698)),Variable(?e),Any), Mapped(Variable(?e),Constant(Constant(52500897)),Variable(?c),Any), Mapped(Variable(?d),Constant(Constant(1496784743)),Any,Any), Mapped(Variable(?c),Constant(Constant(52500897)),Variable(?d),Any), Mapped(Variable(?c),Constant(Constant(217712947)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(-901057765)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(624690160)),AnyConstant,Any)),Some(Mapped(Variable(?a),OneOf(ArrayBuffer(Constant(C

rules: 251
duration: 1s


startTimeMillis: Long = 1635317389625L
alignmentSalariesPatternTaskRuleset: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@3efad6eb
endTimeMillis: Long = 1635317391494L
durationSeconds: Long = 1L

In [56]:
alignmentSalariesPatternTaskRuleset.export("../rulesets/salaries-wikidata/alignmentSalariesPatternTaskRuleset.txt")

In [57]:
alignmentSalariesPatternTaskRuleset
.filter(rule => rule.measures(Measure.Support).value > 1)
.computePcaConfidence(0.0)
.sortBy(Measure.PcaConfidence, Measure.Support)
.export("../rulesets/salaries-wikidata/alignmentSalariesPatternTaskRulesetFiltered.txt")

In [58]:
val partySalariesPattern: RulePattern = (
    AtomPattern(subject = 'a', predicate = czsoRefArea, `object` = 'f') &:
    AtomPattern(subject = 'f', predicate = wdProperty(6), `object` = 'e') &:
    AtomPattern(subject = 'e', predicate = wdProperty(6), `object` = 'd') &:
    AtomPattern(subject = 'd', predicate = wdProperty(102), `object` = 'c') &:  
    AtomPattern(subject = 'c', predicate = wdProperty(102), `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = appliesTo, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = czsoRefPeriod, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = qbDataSet, `object` = AnyConstant)
    =>:
    AtomPattern(subject = 'a', predicate = oneOfSalariesMeasures)
)

partySalariesPattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(
      Variable(Variable(0)),
      Constant(LongUri("http://data.czso.cz/ontology/refArea")),
      Variable(Variable(5)),
      Any
    ),
    AtomPattern(
      Variable(Variable(5)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(4)),
      Any
    ),
    AtomPattern(
      Variable(Variable(4)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(3)),
      Any
    ),
    AtomPattern(
      Variable(Variable(3)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(2)),
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      AnyConstant,
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(
        LongUri("http://kizi.vse.cz/novp19/diploma-thesis/appliesToRefPeriod")
      ),
      Variable(Variable(1)),
   

In [59]:
val partySalariesPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(9))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(1))
    .addPattern(partySalariesPattern)

partySalariesPatternTask: com.github.propi.rdfrules.algorithm.RulesMining = com.github.propi.rdfrules.algorithm.amie.Amie@786c0c61

In [60]:
val startTimeMillis = System.currentTimeMillis()

val partySalariesPatternTaskRuleset = index.mine(partySalariesPatternTask)
println("rules: "+partySalariesPatternTaskRuleset.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

2021-10-27 08:56:03:356 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=1,
MinHeadCoverage=0.0,
MinSupport=1,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=9,
WithConstants=true,
ConstantsPosition=All,
Timeout=60000,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?a),Constant(Constant(1481837794)),Variable(?f),Any), Mapped(Variable(?f),Constant(Constant(774364698)),Variable(?e),Any), Mapped(Variable(?e),Constant(Constant(774364698)),Variable(?d),Any), Mapped(Variable(?d),Constant(Constant(52500897)),Variable(?c),Any), Mapped(Variable(?c),Constant(Constant(52500897)),AnyConstant,Any), Mapped(Variable(?c),Constant(Constant(217712947)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(-901057765)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(624690160)),AnyConstant,Any)),Some(Mapped(Variable(?a),OneOf(ArrayBuffer(Constant(Constant(1433642368)), Constant(Constant(1396113393)))),Any,Any)),f

rules: 296
duration: 1s


startTimeMillis: Long = 1635317763355L
partySalariesPatternTaskRuleset: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@575032da
endTimeMillis: Long = 1635317765092L
durationSeconds: Long = 1L

In [61]:
partySalariesPatternTaskRuleset.export("../rulesets/salaries-wikidata/partySalariesPatternTaskRuleset.txt")

In [62]:
partySalariesPatternTaskRuleset
.filter(rule => rule.measures(Measure.Support).value > 1)
.computePcaConfidence(0.0)
.sortBy(Measure.PcaConfidence, Measure.Support)
.export("../rulesets/salaries-wikidata/partySalariesPatternTaskRulesetFiltered.txt")